In [11]:
# ╭──────────────────────────────────────────────╮
# │ 1. 加载 API Key（从 .env 文件）              │
# │                                              │
# │ %load_ext dotenv + %dotenv 是 Jupyter 魔法命令，│
# │ 用于自动加载 .env 中的环境变量。             │
# ╰──────────────────────────────────────────────╯

%load_ext dotenv
%dotenv

import os
from dotenv import load_dotenv
load_dotenv()  # 兜底：确保在非 Jupyter 环境也能加载

QWEN_API_KEY = os.getenv("QWEN_API_KEY")
ARK_API_KEY = os.getenv("ARK_API_KEY")

if not QWEN_API_KEY:
    raise ValueError("❌ 未找到 QWEN_API_KEY（来自阿里云百炼）")
if not ARK_API_KEY:
    raise ValueError("❌ 未找到 ARK_API_KEY（来自火山方舟）")

print("✅ 环境变量加载成功！")

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
✅ 环境变量加载成功！


In [13]:
# ╭──────────────────────────────────────────────╮
# │ 2. 定义统一 LLM 接口                         │
# │                                              │
# │ 所有模型都继承 BaseLLM，提供一致的 .chat() 方法 │
# ╰──────────────────────────────────────────────╯

from abc import ABC, abstractmethod
from openai import OpenAI

class BaseLLM(ABC):
    @abstractmethod
    def chat(self, messages):
        """发送对话历史，返回模型回复"""
        pass

class QwenLLM(BaseLLM):
    def __init__(self, api_key: str, model: str = "qwen-max"):
        self.client = OpenAI(
            api_key=api_key,
            base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
        )
        self.model = model
    
    def chat(self, messages):
        resp = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            temperature=0.7
        )
        return resp.choices[0].message.content

class DoubaoLLM(BaseLLM):
    def __init__(self, api_key: str, model: str = "doubao-seed-1-8-251228"):
        """
        使用你指定的豆包模型：doubao-seed-1-8-251228
        ⚠️ 请确保已在火山方舟控制台开通该模型！
        """
        self.client = OpenAI(
            api_key=api_key,
            base_url="https://ark.cn-beijing.volces.com/api/v3"
        )
        self.model = model
    
    def chat(self, messages):
        resp = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            temperature=0.7
        )
        return resp.choices[0].message.content

In [14]:
# ╭──────────────────────────────────────────────╮
# │ 3. 初始化模型实例                            │
# │                                              │
# │ 必须执行此 Cell，否则后续会报：               │
# │   NameError: name 'qwen' is not defined      │
# ╰──────────────────────────────────────────────╯

qwen = QwenLLM(QWEN_API_KEY)
doubao = DoubaoLLM(ARK_API_KEY) 

print("✅ Qwen 和 Doubao 模型已成功初始化！")

✅ Qwen 和 Doubao 模型已成功初始化！


In [15]:
# ╭──────────────────────────────────────────────╮
# │ 4. 编排模式 1：串行流水线                    │
# │   Qwen 写初稿 → Doubao 润色                  │
# ╰──────────────────────────────────────────────╯

def draft_and_refine(topic: str):
    """用 Qwen 起草，Doubao 润色"""
    print(f"📝 正在用 Qwen 生成「{topic}」的初稿...")
    draft = qwen.chat([{"role": "user", "content": f"写一篇关于{topic}的短文草稿，100字左右"}])
    
    print("✨ 正在用 Doubao 润色...")
    refined = doubao.chat([
        {"role": "user", "content": f"请将以下文字润色得更流畅、专业，保持原意：\n\n{draft}"}
    ])
    return draft, refined

# 执行编排
draft, final = draft_and_refine("人工智能对教育的影响")

print("\n" + "="*50)
print("📄 最终结果：")
print("初稿：")
print(draft)
print("\n润色后：")
print(final)

📝 正在用 Qwen 生成「人工智能对教育的影响」的初稿...
✨ 正在用 Doubao 润色...

📄 最终结果：
初稿：
人工智能正在深刻改变教育领域，通过个性化学习方案满足不同学生的需求，使得教育更加高效和针对性。智能辅导系统能够提供即时反馈，帮助学生快速掌握知识难点。同时，AI还能辅助教师进行作业批改、学情分析等工作，减轻教师负担，让老师有更多时间关注学生的成长与发展。此外，虚拟现实等技术的应用也为远程教育带来了全新的体验，促进了教育资源的均衡分配。

润色后：
人工智能正深度重塑全球教育格局：依托算法驱动的个性化学习路径，精准匹配不同学生的认知水平、学习节奏与发展需求，让教育供给更具高效性与针对性；智能辅导系统可基于实时学习数据生成即时反馈，精准定位知识盲区，助力学生高效突破学习难点。

与此同时，AI技术深度赋能教师教学全流程：自动完成作业批改、多维度学情分析等重复性事务，有效降低教师的非教学性负担，使其能将更多精力投入到学生的个性化成长引导与全面发展关怀中。

此外，虚拟现实（VR）、增强现实（AR）等沉浸式技术与远程教育的深度融合，打破了地域与场景限制，为学习者打造兼具互动性与沉浸感的学习场景，进一步推动优质教育资源的跨区域均衡配置，助力缩小城乡、区域间的教育差距。


In [16]:
# ╭──────────────────────────────────────────────╮
# │ 5. 编排模式 2：并行调用多个模型              │
# │   同时问 Qwen 和 Doubao，对比回答            │
# ╰──────────────────────────────────────────────╯

from concurrent.futures import ThreadPoolExecutor
import time

def ask_all_models(question: str, models):
    """并行调用多个模型"""
    def _call(model_pair):
        name, model = model_pair
        start = time.time()
        answer = model.chat([{"role": "user", "content": question}])
        latency = time.time() - start
        return name, answer, latency
    
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(_call, models))
    return results

# 执行
question = "如何高效学习大语言模型？"
models = [("Qwen", qwen), ("Doubao", doubao)]

print(f"❓ 问题：{question}\n")
results = ask_all_models(question, models)

for name, answer, latency in results:
    print(f"🤖 {name} ({latency:.2f}s):")
    print(f"   {answer[:120]}...\n")

❓ 问题：如何高效学习大语言模型？

🤖 Qwen (28.74s):
   学习大语言模型（LLM, Large Language Model）是一个既有趣又充满挑战的过程。这类模型基于深度学习技术，能够处理和生成自然语言文本。为了高效地学习这一领域，你可以遵循以下几个步骤：

1. **建立坚实的数学基础**：理...

🤖 Doubao (66.00s):
   学习大语言模型（LLM）需要兼顾**理论基础、实践操作、前沿跟进**三个维度，以下是分阶段的高效学习路径，附核心资源与方法建议：


### **一、入门阶段：搭建知识框架（1-2周）**
目标：快速理解大语言模型的核心概念、发展脉络和应用...



In [18]:
# ╭──────────────────────────────────────────────╮
# │ 6. 编排模式 3：模型路由（Router）            │
# │   根据问题类型自动选择最合适的模型           │
# ╰──────────────────────────────────────────────╯

def smart_router(question: str):
    """
    简单规则路由：
      - 技术/代码类 → Qwen（更强的逻辑与代码能力）
      - 创意/语言类 → Doubao（更流畅的中文表达）
    """
    tech_keywords = ["代码", "编程", "Python", "算法", "调试", "API", "函数"]
    creative_keywords = ["诗歌", "故事", "文案", "润色", "创意", "写作", "春天"]

    if any(kw in question for kw in tech_keywords):
        model_name, model = "Qwen", qwen
        reason = "检测到技术关键词，使用 Qwen"
    elif any(kw in question for kw in creative_keywords):
        model_name, model = "Doubao", doubao
        reason = "检测到创意关键词，使用 Doubao"
    else:
        # 默认用 Doubao（或可改为随机/性能优先）
        model_name, model = "Doubao", doubao
        reason = "无明确关键词，默认使用 Doubao"

    print(f"🔍 路由决策: {reason}")
    answer = model.chat([{"role": "user", "content": question}])
    return model_name, answer

# 测试不同问题
test_questions = [
    "如何用 Python 读取 JSON 文件？",
    "写一首关于秋天的五言诗",
    "解释机器学习是什么"
]

for q in test_questions:
    print(f"\n❓ 问题: {q}")
    model_used, response = smart_router(q)
    print(f"🤖 {model_used}: {response[:100]}...")


❓ 问题: 如何用 Python 读取 JSON 文件？
🔍 路由决策: 检测到技术关键词，使用 Qwen
🤖 Qwen: 在 Python 中读取 JSON 文件通常涉及使用 `json` 模块，该模块提供了简单的方法来解码和编码 JSON 数据。下面是一个基本的步骤指南，展示如何从文件中读取 JSON 数据：

###...

❓ 问题: 写一首关于秋天的五言诗
🔍 路由决策: 无明确关键词，默认使用 Doubao
🤖 Doubao: # 秋夜即景
风疏桐叶坠，露冷菊香清。
雁唳霜空远，灯残客思生。

这首五言绝句以秋夜之景起笔：风过疏林，梧桐叶落，冷露凝枝时菊香愈显清冽；继而以雁唳霜空的苍凉之境，转至残灯之下游子的思乡愁绪，将秋的...

❓ 问题: 解释机器学习是什么
🔍 路由决策: 无明确关键词，默认使用 Doubao
🤖 Doubao: 机器学习是人工智能（AI）的核心分支之一，简单来说，它的目标是让计算机**不用被人类硬编码所有规则**，而是通过「学习数据中的规律」，自主完成预测、分类、决策等任务。

我们可以从几个角度拆解理解：
...


In [19]:
# ╭──────────────────────────────────────────────╮
# │ 7. 编排模式 4：并行投票（Ensemble）          │
# │   多模型回答 → 投票/打分 → 选最优答案         │
# ╰──────────────────────────────────────────────╯

from collections import Counter

def ensemble_voting(question: str, models, top_k=1):
    """
    让多个模型回答同一问题，通过简单投票（或后续可加 LLM 打分）选最佳
    当前简化版：返回所有答案，供人工判断
    """
    answers = []
    
    def get_answer(model_pair):
        name, model = model_pair
        ans = model.chat([{"role": "user", "content": question}])
        return {"model": name, "answer": ans}
    
    with ThreadPoolExecutor() as executor:
        answers = list(executor.map(get_answer, models))
    
    print(f"🗳️  Ensemble 结果（{len(answers)} 个模型）:")
    for i, a in enumerate(answers, 1):
        print(f"  {i}. {a['model']}: {a['answer'][:120]}...")
    
    # TODO: 可接入评分模型（如用 Qwen 给每个答案打分）
    return answers

# 测试
ensemble_voting(
    "如何保持健康的生活方式？",
    [("Qwen", qwen), ("Doubao", doubao)]
)

🗳️  Ensemble 结果（2 个模型）:
  1. Qwen: 保持健康的生活方式对于提高生活质量、延长寿命至关重要。以下是一些有助于维持健康生活方式的建议：

1. **均衡饮食**：确保日常饮食中包含足够的蔬菜、水果、全谷物以及适量的蛋白质（如鱼、肉、豆类）。限制加工食品和高糖、高脂肪的食物摄入。
...
  2. Doubao: 保持健康的生活方式是一个长期的、综合性的习惯养成过程，以下从多个核心维度给出具体建议，帮助你建立适合自己的健康模式：

### 一、均衡饮食：打造营养基础
1. **食物多样化，合理搭配**  
   - 主食优先选全谷物（燕麦、糙米、藜麦...


[{'model': 'Qwen',
  'answer': '保持健康的生活方式对于提高生活质量、延长寿命至关重要。以下是一些有助于维持健康生活方式的建议：\n\n1. **均衡饮食**：确保日常饮食中包含足够的蔬菜、水果、全谷物以及适量的蛋白质（如鱼、肉、豆类）。限制加工食品和高糖、高脂肪的食物摄入。\n\n2. **定期锻炼**：成年人每周至少应进行150分钟的中等强度运动或75分钟的高强度运动。可以选择散步、跑步、游泳、骑自行车等多种形式来增加身体活动量。\n\n3. **充足睡眠**：成年人每晚建议获得7到9小时的高质量睡眠。良好的睡眠习惯有助于恢复体力，改善记忆力，并促进心理健康。\n\n4. **保持水分**：每天喝足够的水对于维持身体健康非常重要。成人每日推荐饮水量约为2-3升，但具体需求可能因个人差异而异。\n\n5. **管理压力**：学会通过冥想、深呼吸或其他放松技巧来缓解生活中的压力。长期处于高压状态会对身心健康产生负面影响。\n\n6. **戒烟限酒**：吸烟和过量饮酒都是导致多种慢性疾病的风险因素。如果难以自行戒除，可以寻求专业帮助。\n\n7. **定期体检**：通过定期进行健康检查，及早发现并处理潜在问题，是维护良好健康状况的关键之一。\n\n8. **积极社交**：与家人朋友保持联系，参与社区活动，建立积极的人际关系网络对精神健康非常有益。\n\n每个人的具体情况不同，因此在调整生活习惯时也应考虑个人的身体条件和生活环境。如有需要，也可以咨询医生或营养师获取更专业的指导。'},
 {'model': 'Doubao',
  'answer': '保持健康的生活方式是一个长期的、综合性的习惯养成过程，以下从多个核心维度给出具体建议，帮助你建立适合自己的健康模式：\n\n### 一、均衡饮食：打造营养基础\n1. **食物多样化，合理搭配**  \n   - 主食优先选全谷物（燕麦、糙米、藜麦、全麦面包等），替代部分精米白面，补充膳食纤维和B族维生素；  \n   - 每天摄入优质蛋白质：鱼、禽、蛋、奶、豆制品（如豆腐、豆浆），红肉（猪牛羊）适量，避免过多加工肉制品（香肠、培根）；  \n   - 保证充足蔬果：每天至少300-500克蔬菜（深色蔬菜占一半，如菠菜、西兰花、紫甘蓝），200-350克水果（选应季低糖品种，如苹果、蓝莓、猕

In [ ]:
# ╭──────────────────────────────────────────────╮
# │ 8. 四种编排模式适用场景总结                  │
# ╰──────────────────────────────────────────────╯

print("📊 LLM 编排模式指南：")
print("\n1️⃣ 串行流水线（Pipeline）")
print("   → 适用：多步骤任务（生成→润色→摘要）")
print("   → 优点：流程清晰；缺点：延迟累加")

print("\n2️⃣ 并行调用（Parallel）")
print("   → 适用：结果对比、A/B 测试")
print("   → 优点：低延迟；缺点：成本翻倍")

print("\n3️⃣ 模型路由（Router）")
print("   → 适用：异构任务（技术 vs 创意）")
print("   → 优点：成本最优；缺点：规则需维护")

print("\n4️⃣ 并行投票（Ensemble）")
print("   → 适用：高可靠性场景（医疗、金融）")
print("   → 优点：鲁棒性强；缺点：成本高、需评分机制")

print("\n💡 真实 Agent 往往混合多种模式！")